In [1]:
import utility_common as common
from utility_dfcutter import *
from utility_dfplotter import *
import glob
binedges = [15,17,20,25,30,40,50]

yields = pd.DataFrame()

In [13]:
# QCD->electron
selection, folderOfPickles = "mumue", "pickles_2016_llx_VTtau"
pickleDirectry = common.getBaseDirectory() + "data/{}/{}/".format(folderOfPickles,selection)
cuts = "lepton1_pt>25 & lepton2_pt>15 & lepton3_pt > 15 & lepton3_mt < 60 & nProbs == 1"
subtitle = r"$Z (\mu \mu)+\tilde{e}$"



In [16]:
# QCD->muon
selection, folderOfPickles = "eemu", "pickles_2016_llx_VTtau"
pickleDirectry = common.getBaseDirectory() + "data/{}/{}/".format(folderOfPickles,selection)
cuts = "lepton1_pt>30 & lepton2_pt>20 & lepton3_pt > 15 & lepton3_mt < 60 & nProbs == 1"
subtitle = r"$Z (e e)+\tilde{\mu}$"

In [6]:
# data
pickles = glob.glob(pickleDirectry + '{}/*.pkl'.format("data"))
data = pd.concat([ pd.read_pickle(pickle) for pickle in pickles],ignore_index=True).query(cuts)
data.reset_index(drop=True, inplace=True)

# top
pickles = glob.glob(pickleDirectry + '{}/*.pkl'.format("mct"))
pickles.append(pickleDirectry+'mctt/ntuple_ttbar_inclusive.pkl')
mct = pd.concat([ pd.read_pickle(pickle) for pickle in pickles],ignore_index=True).query(cuts)
mct.reset_index(drop=True, inplace=True)

# DY
pickles = glob.glob(pickleDirectry + '{}/*.pkl'.format("mcw"))
pickles.append(pickleDirectry+'mcz/ntuple_zjets_m-50_amcatnlo.pkl')
mcz = pd.concat([ pd.read_pickle(pickle) for pickle in pickles],ignore_index=True).query(cuts)
mcz.reset_index(drop=True, inplace=True)


# VV
pickles = []
pickles.append(pickleDirectry+'mcdiboson/ntuple_ww.pkl')
pickles.append(pickleDirectry+'mcdiboson/ntuple_wz_2l2q.pkl')
pickles.append(pickleDirectry+'mcdiboson/ntuple_zz_2l2nu.pkl')
pickles.append(pickleDirectry+'mcdiboson/ntuple_zz_2l2q.pkl')
mcvv = pd.concat([ pd.read_pickle(pickle) for pickle in pickles],ignore_index=True).query(cuts)
mcvv.reset_index(drop=True, inplace=True)

# VV prompt
pickles = []
pickles.append(pickleDirectry+'mcdiboson/ntuple_zz_4l.pkl')
mczz = pd.concat([ pd.read_pickle(pickle) for pickle in pickles],ignore_index=True).query(cuts)
mczz.reset_index(drop=True, inplace=True)

pickles = []
pickles.append(pickleDirectry+'mcdiboson/ntuple_wz_3lnu.pkl')
mcwz = pd.concat([ pd.read_pickle(pickle) for pickle in pickles],ignore_index=True).query(cuts)
mcwz.reset_index(drop=True, inplace=True)

dfList   = [mcwz,mczz,mcvv,mcz,mct,data]
legends  = ['WZ(3l)','ZZ(4l)','VV',"DY","top","data"]

In [7]:
def getYields(dfs, slt):
    w = pd.concat(dfs,ignore_index=True).query(slt).eventWeight
    return sum(w), sum(w**2)

slt = "{}<=lepton3_pt<{}".format(0,1e4)
sltpass = "& lepton3_passiso==1"
sltfail = "& lepton3_passiso==0"

for addslt,addsltTitle in zip([sltpass,sltfail,""],[r",$iso$",r",$\overline{iso}$",""]):
    record = {"channel": subtitle+addsltTitle}
    nmc,nmcvar= 0,0
    for i in range(len(dfList)):
        n, nvar = getYields([dfList[i]], slt+addslt)
        if i == len(dfList)-1:
            record["MC"] = r"${:6.1f} \pm {:6.1f}$".format(nmc,nmcvar**0.5)
        record[legends[i]] = r"${:6.1f} \pm {:6.1f}$".format(n,nvar**0.5)
        nmc += n
        nmcvar += nvar
        
    yields = yields.append(record,ignore_index=True)


In [9]:
yields.to_csv("yields.tex")

In [8]:
yields = yields[['channel','WZ(3l)','ZZ(4l)','VV',"DY","top","MC","data"]]


In [69]:
def getYields(dfs, slt):
    w = pd.concat(dfs,ignore_index=True).query(slt).eventWeight
    return sum(w), sum(w**2)

def getRatio(n,m,nvar,mvar):
    return n/m, (n/m)**2 * (nvar/n**2 + mvar/m**2)    


sltpass = "& lepton3_passiso==1"
sltfail = "& lepton3_passiso==0"

df_ratios = pd.DataFrame()
for i in range(len(binedges)-1):
    a, b = binedges[i], binedges[i+1]
    slt = "{}<=lepton3_pt<{}".format(a,b)

    # pass iso
    n_data, nVar_data = getYields([data],slt+sltpass)
    n_p, nVar_p       = getYields([mcwz,mczz],slt+sltpass)
    n_np, nVar_np     = getYields([mcvv,mct,mcz],slt+sltpass)
    # fail iso
    m_data, mVar_data = getYields([data],slt+sltfail)
    m_p, mVar_p       = getYields([mcwz,mczz],slt+sltfail)
    m_np, mVar_np     = getYields([mcvv,mct,mcz],slt+sltfail)    
    # ratio of pass/fail
    r_data,rVar_data  = getRatio(n_data-n_p, m_data-m_p, nVar_data+nVar_p, mVar_data+mVar_p)
    r_np, rVar_np     = getRatio(n_np, m_np, nVar_np, mVar_np)
    
    df_ratios = df_ratios.append({
        'a':a, 'b':b, 'c':(a+b)/2, 'w':abs(a-b)/2, 
        'n_data':n_data, 'nVar_data':nVar_data,
        'n_p':n_p, 'nVar_p':nVar_p,
        'n_np':n_np, 'nVar_np':nVar_np,
        'm_data':m_data, 'mVar_data':mVar_data,
        'm_p':m_p, 'mVar_p':mVar_p,
        'm_np':m_np, 'mVar_np':mVar_np,
        'r_data':r_data,'rVar_data':rVar_data,
        'r_np':r_np, 'rVar_np':rVar_np }, 
        ignore_index=True)

df_ratios.to_csv("../data/qcdfakes/{}.csv".format(selection))


# make plot

In [17]:
df = pd.read_csv("../data/qcdfakes/{}.csv".format(selection))

In [18]:
df["rStd_data"] = df["rVar_data"]**0.5
df[["a","b","r_data","rStd_data"]]

,a,b,r_data,rStd_data
0,15.0,17.0,0.066638,0.007360
1,17.0,20.0,0.058980,0.007122
2,20.0,25.0,0.046189,0.007531
3,25.0,30.0,0.048913,0.011753
4,30.0,40.0,0.054456,0.014125
5,40.0,50.0,0.109748,0.030272


fig, axes = plt.subplots(2, 1, sharex=True, facecolor='w',
                         gridspec_kw={'height_ratios':[2,1]},
                         figsize=(6,6))
fig.subplots_adjust(hspace=0)
a,b = 12,53

##############
# upper plot
##############
ax = axes[0]
# plot failing events
ax.hist([df.c,df.c], bins=binedges, 
         weights=[df.m_p/df.w/2,df.m_np/df.w/2],color=["C0","C1"],alpha=0.5,
         stacked=True,label=[r"MC ($lep \to \tilde{l}, \overline{iso}$)",r"MC ($jet\to \tilde{l},\overline{iso}$)"])
ax.errorbar(df.c, df.m_data/df.w/2, yerr=df.mVar_data**0.5/df.w/2, fmt='o',markersize=4,color='k',label=r"Data ($\overline{iso}$)")
ystd,y = (df.mVar_p+df.mVar_np)**0.5/df.w/2, (df.m_p+df.m_np)/df.w/2
ax.bar(df.c, 2*ystd, bottom=y-ystd,width=df.w*2*0.7, color='k',alpha=0.3)
ynorm= max(y)
# plot passing events
ax.hist([df.c,df.c], bins=binedges, 
         weights=[df.n_p/df.w/2,df.n_np/df.w/2],color=["C0","C1"],linewidth=3,
         stacked=True,histtype="step",label=[r"MC ($lep \to \tilde{l},iso$)",r"MC ($jet \to \tilde{l},iso$)"])
ax.errorbar(df.c, df.n_data/df.w/2,yerr=df.nVar_data**0.5/df.w/2, fmt='s',markersize=4, color='k',label=r"Data (${iso}$)")
ystd,y = (df.nVar_p+df.nVar_np)**0.5/df.w/2, (df.n_p+df.n_np)/df.w/2
ax.bar(df.c, 2*ystd, bottom=y-ystd,width=df.w*2*0.7, color='k',alpha=0.3)
ynorm= max(ynorm,max(y))

# setting
ax.legend(fontsize=9)
ax.grid(True,linestyle='--',alpha=0.5)
ax.set_yscale("log")
ax.set_ylim(1,10*ynorm)
ax.set_xlim(a,b)
ax.text(0.04*b+0.96*a, 4*ynorm, 
        r'CMS $preliminary$',
        style='italic',fontsize='15',fontweight='bold')
ax.set_title('Run 2016, L=35.9/fb (13TeV)',loc='right')
ax.text(0.04*b+0.96*a,2*ynorm, subtitle)



##############
# lower plot
##############
ax = axes[1]
y,ystd = df.r_np, df.rVar_np**0.5
# ax.bar(df.c, 2*ystd, bottom=y-ystd, width= 2*df.w, color="C1", alpha=0.2)
ax.errorbar(df.c, y, yerr=ystd, xerr=df.w, color="C1", fmt=".", alpha=1)

y,ystd = df.r_data, df.rVar_data**0.5
ax.errorbar(df.c, y, yerr=ystd, xerr=df.w, color="k", fmt=".")
ax.bar(df.c, 2*ystd, bottom=y-ystd, width= 2*df.w, color="k", alpha=0.3)
for i in np.arange(3,len(y)):
    ax.text(df.c[i], y[i]+0.1*ynormSF, r"${:4.3}\pm{:4.2}$".format(y[i],ystd[i]), fontsize=9, ha="center", va="bottom")


ax.grid(True,linestyle='--',alpha=0.5)

ax.set_ylim(0,ynormSF)
ax.set_xlabel(configs[4][-1])
ax.text(0.04*b+0.96*a,0.7*ynormSF,r"SF = $\frac{P(jet \to \tilde{l},iso )} {P(jet \to \tilde{l}, \overline{iso})} $",fontsize=12,
       ha="left", va="center",bbox=dict(boxstyle="round",
                   ec='k',fc="w",
                   ))
ax.set_ylabel("SF")
plt.savefig("../plots/kinematics/{}/sf.pdf".format(selection))
plt.savefig("../plots/kinematics/{}/sf.png".format(selection),dpi=300)